In [1]:
import pandas as pd
import sqlite3

In [2]:
volcanoes_df = pd.read_csv('../csv/cleaned/volcanoes_cleaned.csv')
volcanoes_df.columns = volcanoes_df.columns.str.replace(' ','_').str.lower()

In [3]:
eruptions_df = pd.read_csv('../csv/cleaned/eruptions_cleaned.csv')
eruptions_df.columns = eruptions_df.columns.str.replace(' ','_').str.lower()

In [4]:
# Connect to sqlite db and create tables if they don't exist
cnx = sqlite3.connect('volcanoes.db')
volcanoes_df.to_sql(name='volcanoes', con=cnx, if_exists='replace')
eruptions_df.to_sql(name='eruptions', con=cnx, if_exists='replace')

### All years in eruptions, considering both start_year and end_year

In [5]:
all_years_eruptions_query = """
    WITH all_years AS ( SELECT 
    start_year AS eruption_year
    FROM ( SELECT 
            start_year
            FROM
                eruptions
            WHERE
                start_year >= 1900
            UNION 
            SELECT 
                end_year
            FROM
                eruptions
            WHERE
                end_year >= 1900
            ) ORDER BY eruption_year
)
SELECT 
    eruption_year,
    SUM(start_year = ay.eruption_year) AS started_in_year_count,
    SUM(end_year = ay.eruption_year) AS ended_in_year_count,
    COUNT(*) AS total_eruptions
FROM
    all_years ay
        LEFT JOIN
    eruptions e ON ay.eruption_year BETWEEN e.start_year AND e.end_year
GROUP BY ay.eruption_year;
"""

all_years_eruptions_df = pd.read_sql_query(all_years_eruptions_query, cnx)
all_years_eruptions_df

,eruption_year,started_in_year_count,ended_in_year_count,total_eruptions
0,1900.0,12,13,27
1,1901.0,8,10,22
2,1902.0,22,16,34
3,1903.0,14,14,32
4,1904.0,19,20,37
...,...,...,...,...
116,2016.0,35,43,84
117,2017.0,35,35,76
118,2018.0,39,35,80
119,2019.0,29,30,74


### Top 20 volcanoes with most eruptions since 1900 


In [6]:
most_eruptions_query = """ 
SELECT 
    COUNT(e.eruption_number) eruptions_count,
    v.volcano_name || ' (' || v.country || ')' AS volcano_name
FROM
    eruptions e
        JOIN
    volcanoes v ON e.volcano_number = v.volcano_number
WHERE
    e.start_year > 1900
GROUP BY v.volcano_name, v.country
ORDER BY eruptions_count DESC
LIMIT 20;
"""

most_eruptions_df = pd.read_sql_query(most_eruptions_query, cnx)
most_eruptions_df

,eruptions_count,volcano_name
0,94,"Fournaise, Piton de la (France)"
1,83,Etna (Italy)
2,77,Asosan (Japan)
3,73,Klyuchevskoy (Russia)
4,57,Ruapehu (New Zealand)
5,53,Kilauea (United States)
6,52,Asamayama (Japan)
7,52,Bezymianny (Russia)
8,49,Raung (Indonesia)
9,48,Karangetang (Indonesia)


### Top 20 longest eruptions

In [7]:
longest_eruptions_query = """
WITH eruptions_with_dates AS ( SELECT 
    PRINTF('%02d',CAST(e.start_year AS INTEGER)) || "-" ||
    PRINTF('%02d',CAST(e.start_month AS INTEGER)) || "-" ||
    PRINTF('%02d',CAST(e.start_day AS INTEGER)) AS start_date,
    PRINTF('%02d',CAST(e.end_year AS INTEGER)) || "-" ||
    PRINTF('%02d',CAST(e.end_month AS INTEGER)) || "-" ||
    PRINTF('%02d',CAST(e.end_day AS INTEGER)) AS end_date,           
    v.volcano_name,
    v.country,
    v.primary_volcano_type
    FROM eruptions e 
    JOIN
    volcanoes v ON e.volcano_number = v.volcano_number
    WHERE start_month > 0 AND start_day>0 AND end_month>0 and end_day>0 
)
SELECT *, (JULIANDAY(end_date) - JULIANDAY(start_date)) AS eruption_days_length 
FROM eruptions_with_dates
ORDER BY eruption_days_length DESC
LIMIT 20;
"""

longest_eruptions_df = pd.read_sql_query(longest_eruptions_query, cnx)
longest_eruptions_df

,start_date,end_date,volcano_name,country,primary_volcano_type,eruption_days_length
0,1774-07-02,2020-02-20,Yasur,Vanuatu,Stratovolcano,89717.0
1,1922-06-22,2020-02-19,Santa Maria,Guatemala,Stratovolcano,35671.0
2,1933-08-13,2020-02-19,Dukono,Indonesia,Complex,31601.0
3,1934-02-02,2020-02-20,Stromboli,Italy,Stratovolcano,31429.0
4,1934-08-08,2011-03-02,Sangay,Ecuador,Stratovolcano,27965.0
5,1955-10-13,2016-08-22,Aira,Japan,Caldera,22229.0
6,1967-07-02,2020-02-20,Erta Ale,Ethiopia,Shield,19226.0
7,1927-03-16,1977-01-10,Nyiragongo,DR Congo,Stratovolcano,18198.0
8,1972-12-16,2020-02-18,Erebus,Antarctica,Stratovolcano,17230.0
9,1968-07-29,2010-12-16,Arenal,Costa Rica,Stratovolcano,15480.0


We save the output of queries to CSV files, in order to use them for Tableau visualisations.

In [8]:
all_years_eruptions_df.to_csv('../csv/tableau/all_years_eruptions.csv', index=False)
most_eruptions_df.to_csv('../csv/tableau/volcanoes_most_eruptions.csv', index=False)
longest_eruptions_df.to_csv('../csv/tableau/longest_eruptions.csv', index=False)